In [1]:
PROJECT_ID = "lramsey-dev"
REGION = "us-central1"
import os
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["PG_HOST"] = ""

In [2]:
import sys
sys.path.append("../../common/src")
sys.path.append("../src")

In [3]:
import tempfile
import os
from typing import List, Optional, Tuple, Dict
from common.utils.logging_handler import Logger
from common.models import (UserQuery, QueryResult, QueryEngine, QueryDocument,
                           QueryReference, QueryDocumentChunk, BatchJobModel)
from common.utils.errors import (ResourceNotFoundException,
                                 ValidationError)
from common.utils.http_exceptions import InternalServerError
from utils.errors import NoDocumentsIndexedException
from google.cloud import storage
from services import llm_generate, embeddings
from services.query import query_prompts
from services.query.vector_store import VectorStore
from services.query.data_source import DataSource
from config import (PROJECT_ID, DEFAULT_QUERY_CHAT_MODEL,
                        DEFAULT_QUERY_EMBEDDING_MODEL)
from services.query.query_service import query_generate

INFO: [config/config.py:61 - <module>()] Namespace File not found, setting job namespace as default


/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:173: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:343: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.
  warn_de

INFO: [config/config.py:112 - <module>()] ENABLE_GOOGLE_LLM = True
INFO: [config/config.py:113 - <module>()] ENABLE_OPENAI_LLM = True
INFO: [config/config.py:114 - <module>()] ENABLE_COHERE_LLM = True
INFO: [config/config.py:115 - <module>()] ENABLE_GOOGLE_MODEL_GARDEN = True
INFO: [config/config.py:116 - <module>()] ENABLE_TRUSS_LLAMA2 = False
INFO: [config/config.py:117 - <module>()] ENABLE_VLLM_GEMMA = True
INFO: [config/vector_store_config.py:40 - <module>()] Default vector store = [matching_engine]
INFO: [config/vector_store_config.py:49 - <module>()] PG_HOST = [None]
INFO: [config/vector_store_config.py:50 - <module>()] PG_DBNAME = [pgvector]
INFO: [config/vector_store_config.py:79 - <module>()] PG_HOST is set to [None], not connecting to pgvector
INFO: [config/onedrive_config.py:30 - <module>()] ONEDRIVE_CLIENT_ID = [None]
INFO: [config/onedrive_config.py:31 - <module>()] ONEDRIVE_TENANT_ID = [None]
INFO: [utils/text_helper.py:43 - <module>()] using default spacy model
Loading d

In [4]:
storage_client = storage.Client(project=PROJECT_ID)

In [5]:
qe_list = QueryEngine.fetch_all()
[(qe.name, qe.id) for qe in qe_list]

[('Test LLM Service 2', 'JmKH0Rnt6khHCaRfuSG7'),
 ('Test LLM Service 1', 'SaLOGu4JCvStmK7RFCMD'),
 ('Test Vertex Search 8', 'urjshigECiGqhj90MjJx'),
 ('Test Vertex Search 7', 'BY5hPEunjRYutJVD7DQa'),
 ('Test Vertex Search 6', 'vnoCpT38aKjEZlDjmhVs'),
 ('Test Vertex Search 5', 'WRDCFzLMqJFy8cvDh2WA'),
 ('APG Provider Manual', 'TXfYVDTqotmlF3thAx0O'),
 ('CHCANYS Conference web page from NYSDOH site', 'ZCfGNtk9WuIxgp8wLc6h'),
 ('FQHC Dental Policies from NYSDOH web site', 'x6UkpTjcgVZxCB8kucqU'),
 ('Medicaid dental billing policies', 'dwQL5Ir0iXDpMW4kUEM4')]

In [6]:
query_engine = "Test LLM Service 2"
q_engine = QueryEngine.find_by_name(query_engine)
user_id = "epKVwPZeIBz1rrqcDm1B"

In [7]:
q_engine.get_fields()

{'name': 'Test LLM Service 2',
 'query_engine_type': 'qe_llm_service',
 'description': '',
 'llm_type': 'VertexAI-Chat',
 'embedding_type': 'VertexAI-Embedding',
 'vector_store': 'langchain_pgvector',
 'created_by': 'Z4lrklOGmEUFc8o6KFYavpoROV43',
 'is_public': True,
 'index_id': None,
 'index_name': None,
 'endpoint': None,
 'doc_url': 'https://dmv.nv.gov/',
 'agents': [''],
 'parent_engine_id': None,
 'params': {'depth_limit': 1, 'agents': ''},
 'created_time': DatetimeWithNanoseconds(2024, 3, 17, 15, 33, 33, 918691, tzinfo=datetime.timezone.utc),
 'last_modified_time': DatetimeWithNanoseconds(2024, 3, 17, 15, 33, 34, 51883, tzinfo=datetime.timezone.utc),
 'deleted_at_timestamp': None,
 'deleted_by': '',
 'archived_at_timestamp': None,
 'archived_by': '',
 'last_modified_by': '',
 'id': 'JmKH0Rnt6khHCaRfuSG7'}

In [8]:
prompt = "how do I apply for a commercial license"

In [10]:
_, embeddings = await embeddings.get_embeddings([prompt], q_engine.embedding_type)

INFO: [services/embeddings.py:65 - get_embeddings()] generating embeddings with VertexAI-Embedding
INFO: [services/embeddings.py:149 - generate_embeddings()] generating embeddings for embedding type VertexAI-Embedding
INFO: [config/model_config.py:483 - get_provider_value()] Get provider value:
INFO: [config/model_config.py:484 - get_provider_value()] provider_id=Vertex
INFO: [config/model_config.py:485 - get_provider_value()] model_id=VertexAI-Embedding
INFO: [services/embeddings.py:208 - get_vertex_embeddings()] generating Vertex embeddings for 1 chunk(s) embedding model text-embedding-004


In [11]:
embeddings[0]

array([-2.35764664e-02,  3.82136740e-02,  3.40655185e-02, -3.70504707e-03,
        2.45017521e-02,  2.19221245e-02,  6.38145804e-02,  7.36327097e-03,
        4.91844639e-02,  4.90183644e-02, -2.25496870e-02,  1.85602903e-02,
       -1.37316398e-02,  2.21355148e-02, -3.84428687e-02, -3.92883122e-02,
       -5.87003343e-02,  1.37411542e-02, -1.54102713e-01, -6.28443584e-02,
        4.73594153e-03,  1.92725826e-02, -6.82127327e-02, -3.02298628e-02,
        5.05838916e-02, -4.05319734e-03,  6.60265908e-02, -6.76689968e-02,
        1.99233857e-03, -6.41971603e-02, -2.54400726e-03,  3.74183096e-02,
       -3.03676724e-02,  6.96961209e-03,  8.84275045e-03,  1.64558962e-02,
        2.57458780e-02, -1.08271493e-02,  2.95430943e-02, -6.82244226e-02,
       -6.03051409e-02,  3.15777548e-02, -4.40741889e-03,  3.80443670e-02,
       -7.58454157e-03, -3.67716178e-02, -4.32687812e-03,  8.78752209e-03,
       -6.83041289e-02, -5.71168214e-03,  1.07218862e-01,  1.30796246e-02,
       -2.52754260e-02,  

In [14]:
result = await query_generate(user_id, prompt, q_engine)

INFO: [query/query_service.py:112 - query_generate()] Executing query: llm_type=[None], user_id=[epKVwPZeIBz1rrqcDm1B], prompt=[how do I apply for a commercial license], q_engine=[Test LLM Service 2], user_query=[None]
INFO: [query/query_service.py:299 - query_search()] Retrieving doc references for q_engine=[Test LLM Service 2], query_prompt=[how do I apply for a commercial license]
INFO: [services/embeddings.py:65 - get_embeddings()] generating embeddings with VertexAI-Embedding
INFO: [services/embeddings.py:148 - generate_embeddings()] generating embeddings for embedding type VertexAI-Embedding
INFO: [config/model_config.py:483 - get_provider_value()] Get provider value:
INFO: [config/model_config.py:484 - get_provider_value()] provider_id=Vertex
INFO: [config/model_config.py:485 - get_provider_value()] model_id=VertexAI-Embedding


IndexError: list index out of range